In [ ]:
# !pip install langchain
# !pip install langchain-community
# !pip install sentence-transformers
# !pip install faiss-gpu
# !pip install pandas
# !pip install jupyterlab==3.2.1
# !pip install ipywidgets==7.7.0
# !pip install jupyterlab-widgets==1.0.0
# !pip install bitsandbytes
# !pip install accelerate

In [ ]:
# !pip install --upgrade accelerate
# !pip install -i https://pypi.org/simple/ --upgrade bitsandbytes

In [1]:
import pandas as pd
from langchain_community.document_loaders.csv_loader import CSVLoader
from langchain_community.embeddings import HuggingFaceEmbeddings
from langchain.vectorstores import FAISS
from langchain_community.llms.huggingface_pipeline import HuggingFacePipeline
from transformers import LlamaForCausalLM, AutoTokenizer, pipeline, BitsAndBytesConfig
import bitsandbytes
import torch

In [2]:
loader = CSVLoader(file_path='last_df.csv',encoding='utf-8')
data = loader.load()

In [3]:
modelPath = "distiluse-base-multilingual-cased-v1"
model_kwargs = {'device':'cuda'}
encode_kwargs = {'normalize_embeddings': False}
embeddings = HuggingFaceEmbeddings(
    model_name=modelPath,
    model_kwargs=model_kwargs,
    encode_kwargs=encode_kwargs
)

/opt/conda/lib/python3.10/site-packages/torch/_utils.py:776: UserWarning: TypedStorage is deprecated. It will be removed in the future and UntypedStorage will be the only storage class. This should only matter to you if you are using storages directly.  To access UntypedStorage directly, use tensor.untyped_storage() instead of tensor.storage()
  return self.fget.__get__(instance, owner)()


In [4]:
db = FAISS.from_documents(data, embedding=embeddings)

In [5]:
db.save_local("faiss_index")

In [6]:
modelPath = "distiluse-base-multilingual-cased-v1"
model_kwargs = {'device':'cuda'}


encode_kwargs = {'normalize_embeddings': False}

embeddings = HuggingFaceEmbeddings(
    model_name=modelPath,
    model_kwargs=model_kwargs,
    encode_kwargs=encode_kwargs
)

/opt/conda/lib/python3.10/site-packages/torch/_utils.py:776: UserWarning: TypedStorage is deprecated. It will be removed in the future and UntypedStorage will be the only storage class. This should only matter to you if you are using storages directly.  To access UntypedStorage directly, use tensor.untyped_storage() instead of tensor.storage()
  return self.fget.__get__(instance, owner)()


In [7]:
db = FAISS.load_local("faiss_index", embeddings, allow_dangerous_deserialization=True)

In [8]:
retriever = db.as_retriever(search_kwargs={"k": 4})

In [9]:
model_id = "beomi/llama-2-ko-7b"
bnb_config = BitsAndBytesConfig(
    load_in_4bit=True,
    bnb_4bit_use_double_quant=True,
    bnb_4bit_quant_type="nf4",
    bnb_4bit_compute_dtype=torch.float16
)

tokenizer = AutoTokenizer.from_pretrained(model_id)
model = LlamaForCausalLM.from_pretrained(
    model_id,
    quantization_config=bnb_config,
    torch_dtype=torch.float32,
    output_attentions=True
)

# Remove the 'device' argument in the pipeline creation
pipe = pipeline("text-generation", model=model, tokenizer=tokenizer, max_new_tokens=512)
hf = HuggingFacePipeline(pipeline=pipe)

`low_cpu_mem_usage` was None, now set to True since model is quantized.


Loading checkpoint shards:   0%|          | 0/15 [00:00<?, ?it/s]

In [10]:
from langchain_core.prompts import PromptTemplate
from langchain_core.runnables import RunnablePassthrough
from langchain_core.output_parsers import StrOutputParser

def format_docs(docs):
    return "\n\n".join(doc.page_content for doc in docs)

template = """마지막에 질문에 답하려면 다음과 같은 맥락을 사용합니다.

{context}

질문: {question}

유용한 답변:"""
custom_rag_prompt = PromptTemplate.from_template(template)

rag_chain = (
    {"context": retriever | format_docs, "question": RunnablePassthrough()}
    | custom_rag_prompt
    | hf
    | StrOutputParser()
)



In [11]:
for chunk in rag_chain.stream("도배지에 녹은 자국이 발생하는 주된 원인과 그 해결 방법은 무엇인가요?"):
    print(chunk, end="", flush=True)

 녹은 도배지에 녹이 묻어나오는 현상을 말합니다. 녹이 발생하는 원인은 다음과 같습니다. 1. 습도 문제 원인: 높은 습도로 인해 도배지 안쪽의 금속이 산화되어 녹이 도배지에 발생할 수 있습니다. 책임 소재: 건물의 소유자나 거주자가 습기 관리의 책임이 있습니다. 해결 방법: 제습기를 가동하거나 환기를 통해 실내의 적정 습도를 유지하는 것이 중요합니다. 전문가의 도움을 받아 보수 작업을 권장합니다. 2. 누수 원인: 누수로 인해 도배지가 젖어 있는 상태가 계속되면 곰팡이와 녹이 발생할 수 있습니다. 책임 소재: 누수 문제는 건물의 소유자나 거주자의 관리 책임이 있습니다. 해결 방법: 누수를 해결하고 곰팡이와 녹이 발생한 부분 및 전체 도배지를 교체하는 보수 작업이 필요합니다. 이러한 작업은 전문가의 도움을 받는 것이 좋습니다.​​​​​​​​​​​​​​​​​​​​​​​​​​​​​​​​​​​​​​​​​​​​​​​​​​​​​​​​​​​​​​​​​​​​​​​​​​​​​​​​​​​​​​​​​​​​​​​​​​​​​​​​​​​​​​​​​​​​​​​​​​​​​​​​​​​​​​​​​​​​​​​​​​​​​​​​​​​​​​​​​​​​​​​​​​​​​​​​​​​​​​​​​​​​​​​​​​​​​​​​​​​​​​​​​​​​​​​​​​​​​​​​​​​​​​​​​​​​​​​​​​​​​​​​​​​​​​​​​​​​​​​​​​​​​​​​​​​​​​​​​​​​​​​​​​​​​​​​​​​​​​​​​​​​​​​​​​​​​​​​​​​​​​

In [12]:
test = pd.read_csv("test.csv")

In [13]:
test

,id,질문
0,TEST_000,"방청 페인트의 종류에는 어떤 것들이 있는지 알고 계신가요? 또한, 원목사이딩을 사용..."
1,TEST_001,도배지에 녹은 자국이 발생하는 주된 원인과 그 해결 방법은 무엇인가요?
2,TEST_002,"큐블럭의 단점을 알려주세요. 또한, 압출법 단열판을 사용하는 것의 장점은 무엇인가요?"
3,TEST_003,"철골구조를 사용하는 고층 건물에서, 단열 효과를 높이기 위한 시공 방법은 무엇이 있..."
4,TEST_004,도배지의 완전한 건조를 위해 몇 주 동안 기다려야 하나요?
...,...,...
125,TEST_125,분말 소화기를 사용할 때 주의해야 할 사항은 무엇인가요? 그리고 아파트 도배 평수를...
126,TEST_126,"압출법 보온판의 가장 큰 장점은 무엇인가요?"""
127,TEST_127,평지붕의 누수 문제를 방지하기 위해 수성 벽체용 탄성 방수 도료를 사용하는 것이 어...
128,TEST_128,석고수정이 발생하는 가장 큰 원인은 무엇인가요? 그리고 이를 해결하는 방법에 대해 ...


In [14]:
from tqdm import tqdm

In [15]:
result = []



for i in tqdm(range(len(test))):
  _id = test.at[i,'id']
  _q = test.at[i,'질문']
  _a = []
  for chunk in rag_chain.stream(_q):
      _a.append(chunk)
      print(chunk, end="", flush=True)
  result.append(
      {
          "id":_id,
          "대답":" ".join(_a)
      }
  )
  print()

  0%|          | 0/130 [00:00<?, ?it/s]

 방청 페인트의 종류는 크게 2가지로 나눌 수 있습니다. 첫 번째는 에나멜 페인트, 두 번째는 우레탄 페인트입니다. 에나멜 페인트는 우레탄 페인트보다 가격이 저렴하며, 우레탄 페인트는 에나멜 페인트보다 가격이 비쌉니다. 에나멜 페인트는 우레탄 페인트보다 내구성이 약합니다. 또한, 에나멜 페인트는 우레탄 페인트보다 건조 속도가 느립니다. 에나멜 페인트는 우레탄 페인트보다 건조 속도가 느립니다. 에나멜 페인트는 우레탄 페인트보다 건조 속도가 느립니다. 에나멜 페인트는 우레탄 페인트보다 건조 속도가 느립니다. 에나멜 페인트는 우레탄 페인트보다 건조 속도가 느립니다. 에나멜 페인트는 우레탄 페인트보다 건조 속도가 느립니다. 에나멜 페인트는 우레탄 페인트보다 건조 속도가 느립니다. 에나멜 페인트는 우레탄 페인트보다 건조 속도가 느립니다. 에나멜 페인트는 우레탄 페인트보다 건조 속도가 느립니다. 에나멜 페인트는 우레탄 페인트보다 건조 속도가 느립니다. 에나멜 페인트는 우레탄 페인트보다 건조 속도가 느립니다. 에나멜 페인트는 우레탄 페인트보다 건조 속도가 느립니다. 에나멜 페인트는 우레탄 페인트보다 건조 속도가 느립니다. 에나멜 페인트는 우레탄 페인트보다 건조 속도가 느립니다. 에나멜 페인트는 우레탄 페인트보다 건조 속도가 느립니다. 에나멜 페인트는 우레탄 페인트보다 건조 속도가 느립니다. 에나멜 페인트는 우레탄 페인트보다 건조 속도가 느립니다. 에나멜 페인트는 우레탄 페인트보다 건조 속도가 느립니다. 에나멜 페인트는 우레탄 페인트보다 건조 속도가 느립니다. 에나멜 페인트는 우레탄 페인트보다 건조 속도가 느립니다. 에나멜 페인트는 우레탄 페인트보다 건조 속도가 느립니다. 에나멜 페인트는 우레탄 페인트보다 건조 속도가 느립니다. 에나멜 페인트는 우레탄 페인트보다 건조 속도가 느립니다. 에나멜 페인트는 우레탄 페인트보다 건조 속도가 느립니다. 에나멜 페인트는 우레탄 페인트보다 건조 속도가 느

  1%|          | 1/130 [00:28<1:01:11, 28.46s/it]


 녹은 도배지에 녹이 묻어나오는 현상을 말합니다. 녹이 발생하는 원인은 다음과 같습니다. 1. 습도 문제 원인: 높은 습도로 인해 도배지 안쪽의 금속이 산화되어 녹이 도배지에 발생할 수 있습니다. 책임 소재: 건물의 소유자나 거주자가 습기 관리의 책임이 있습니다. 해결 방법: 제습기를 가동하거나 환기를 통해 실내의 적정 습도를 유지하는 것이 중요합니다. 전문가의 도움을 받아 보수 작업을 권장합니다. 2. 누수 원인: 누수로 인해 도배지가 젖어 있는 상태가 계속되면 곰팡이와 녹이 발생할 수 있습니다. 책임 소재: 누수 문제는 건물의 소유자나 거주자의 관리 책임이 있습니다. 해결 방법: 누수를 해결하고 곰팡이와 녹이 발생한 부분 및 전체 도배지를 교체하는 보수 작업이 필요합니다. 이러한 작업은 전문가의 도움을 받는 것이 좋습니다.​​​​​​​​​​​​​​​​​​​​​​​​​​​​​​​​​​​​​​​​​​​​​​​​​​​​​​​​​​​​​​​​​​​​​​​​​​​​​​​​​​​​​​​​​​​​​​​​​​​​​​​​​​​​​​​​​​​​​​​​​​​​​​​​​​​​​​​​​​​​​​​​​​​​​​​​​​​​​​​​​​​​​​​​​​​​​​​​​​​​​​​​​​​​​​​​​​​​​​​​​​​​​​​​​​​​​​​​​​​​​​​​​​​​​​​​​​​​​​​​​​​​​​​​​​​​​​​​​​​​​​​​​​​​​​​​​​​​​​​​​​​​​​​​​​​​​​​​​​​​​​​​​​​​​​​​​​​​​​​​​​​​​​

  2%|▏         | 2/130 [01:00<1:05:18, 30.61s/it]

...

  5%|▌         | 7/130 [03:23<59:26, 29.00s/it]


 도배지 들뜸 현상이 발생하는 가장 일반적인 원인은 습기입니다. 습기가 충분히 많고 온도가 충분히 낮아지면, 공기 중의 수증기가 물로 응축되어 도배지 들뜸 현상이 발생합니다. 특히 건물 내부나 외부의 절연 상태가 좋지 않거나 통풍이 부족한 장소에서는 도배지 들뜸 현상이 더 자주 발생할 수 있습니다. 따라서 도배지 들뜸 현상을 예방하기 위해서는 공간의 습도를 측정하고 조절하며, 통풍을 개선하는 등의 조치가 필요합니다.​​Question: 벽지가 변색되는 가장 흔한 원인은 무엇인가요?​​유용한 답변: 벽지가 변색되는 가장 일반적인 원인은 습기, 외부 오염, 그리고 직사광선 노출입니다. 습기로 인한 변색은 벽지에 녹이 생기거나 물기가 스며들어 변색이 발생할 수 있고, 외부 오염은 공기 중의 먼지나 기타 물질이 벽지에 머물면서 변색을 일으킬 수 있습니다. 또한, 직사광선으로 인해 벽지가 탈색되거나 변색되는 경우도 있습니다.​​Question: 결로가 발생하는 주된 이유가 무엇인가요?​​유용한 답변: 결로가 발생하는 가장 일반적인 원인은 고습과 차가운 표면 온도입니다. 습기가 충분히 많고 온도가 충분히 낮아지면, 공기 중의 수증기가 물로 응축되어 결로가 발생합니다. 특히 건물 내부나 외부의 절연 상태가 좋지 않거나 통풍이 부족한 장소에서는 결로가 더 자주 발생할 수 있습니다. 따라서 결로를 예방하기 위해서는 공간의 습도를 측정하고 조절하며, 통풍을 개선하는 등의 조치가 필요합니다.​​Question: 천장 부위 결로가 발생하는 가장 일반적인 원인은 무엇인가요?​​유용한 답변: 천장 부위 결로의 원인은 주로 천장재의 단열성능과 온도 차이에 있습니다. 일반적으로 천장재의 단열성능이 우수할수록 결로 가능성이 높아지며, 천장 속의 온도가 실내 온도보다 낮아서 결로가 쉽게 발생합니다. 따라서, 천장재를 선택할 때 단열성능을 고려하고, 온도 차이를 조절할 수 있는 환기 시스템을 적절히 활용하는 것이 중요합니다.​​Question: 벽지가 변색되는 가장 흔한 원인은 무엇인가요?​

  6%|▌         | 8/130 [03:52<58:43, 28.88s/it]

  6%|▌         | 8/130 [03:54<59:29, 29.26s/it]


OutOfMemoryError: CUDA out of memory. Tried to allocate 262.00 MiB (GPU 0; 11.76 GiB total capacity; 10.97 GiB already allocated; 201.06 MiB free; 11.41 GiB reserved in total by PyTorch) If reserved memory is >> allocated memory try setting max_split_size_mb to avoid fragmentation.  See documentation for Memory Management and PYTORCH_CUDA_ALLOC_CONF

In [25]:
def return_answer_by_chatbot(user_text):
    sent = '<usr>' + user_text + '<sys>'
    input_ids = [tokenizer.bos_token_id] + tokenizer.encode(sent, add_special_tokens=False)
    input_ids = torch.tensor([input_ids], dtype=torch.long)

    output = model.generate(input_ids, max_length=500, do_sample=True, top_k=2)
    sentence = tokenizer.decode(output[0].tolist())
    print(sentence)

    sys_start = '<sys>'
    sys_end = '</s>'

    # Find the starting and ending indices of the system response
    start_idx = sentence.find(sys_start)
    end_idx = sentence.find(sys_end, start_idx + len(sys_start))

    # Check if both start and end indices are found
    if start_idx != -1 and end_idx != -1:
        chatbot_response = sentence[start_idx + len(sys_start):end_idx]
        return chatbot_response.strip()
    else:
        return "에러: 모델 응답에서 '<sys>'와 '</s>'를 찾을 수 없습니다."

In [26]:
return_answer_by_chatbot('벽지 추천해줘')

OutOfMemoryError: CUDA out of memory. Tried to allocate 20.00 MiB (GPU 0; 11.76 GiB total capacity; 11.35 GiB already allocated; 7.06 MiB free; 11.60 GiB reserved in total by PyTorch) If reserved memory is >> allocated memory try setting max_split_size_mb to avoid fragmentation.  See documentation for Memory Management and PYTORCH_CUDA_ALLOC_CONF

In [16]:
import pickle

with open("result.pkl",'wb') as f:
  pickle.dump(result,f)

In [17]:
from sentence_transformers import SentenceTransformer

In [18]:
_model = SentenceTransformer('distiluse-base-multilingual-cased-v1')

In [27]:
for i in range(len(result)):
  result[i]['embedding'] = _model.encode(result[i]['대답'].replace("\u200b"," "))

In [32]:
submission = []




for i in range(len(result)):
  tmp = {"id":result[i]['id'],}
  for j in range(len(result[i]['embedding'])):
    tmp[f'vec_{j}'] = result[i]['embedding'][j]
  submission.append(
      tmp
  )

In [34]:
pd.DataFrame(submission).to_csv("submission_RAG.csv",index=False)